<a href="https://colab.research.google.com/github/skywalker0803r/works/blob/master/MultiOutputRegressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip install catboost

In [2]:
from google.colab import drive
drive.mount('/content/drive')
from sklearn.multioutput import MultiOutputRegressor
import pandas as pd
import numpy as np
import warnings;warnings.simplefilter('ignore')
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import BayesianRidge, LinearRegression,Lasso,Ridge
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from math import sqrt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# some function

In [0]:
def metric(y_t,y_p,name):
  res = {'R2':r2_score(y_t,y_p),
         'RMSE':sqrt(mean_squared_error(y_t,y_p)),
         'mean_absolute_error':mean_absolute_error(y_t,y_p),
         'norm_RMSE' : sqrt(mean_squared_error(y_t,y_p))/np.mean(y_t),
         'y_mean' : np.mean(y_t),
         'std' : np.std(y_t),
         }
  return pd.DataFrame(res,index=[name])

In [0]:
def modeling(my_model,train,test):
  model = MultiOutputRegressor(my_model)
  model.fit(train[features],train[y_columns])
  y_pred = model.predict(test[features])
  y_pred = pd.DataFrame(y_pred,columns=y_columns)
  res = pd.DataFrame()
  for y_name in y_columns:
    res = res.append(metric(test[y_name],y_pred[y_name],y_name))
  res.loc['AVG'] = res.mean()
  return res

# load

In [5]:
data = pd.read_csv('/content/drive/My Drive/台塑輕油案子/data/test_250.csv',index_col=0)
train = data.loc[data.index %5 != 0]
test = data.loc[data.index %5 == 0]
print(train.shape)
print(test.shape)

(200, 27)
(50, 27)


# features , y_columns

In [0]:
cols = train.columns.tolist()
features = cols[:4]
y_columns = cols[4:]

# scaler

In [0]:
scaler_x = StandardScaler()
train[features] = scaler_x.fit_transform(train[features])
test[features] = scaler_x.transform(test[features])

# modeling

In [8]:
modeling(SVR(),train,test)

,R2,RMSE,mean_absolute_error,norm_RMSE,y_mean,std
C5NP,0.966166,0.735723,0.526793,0.175238,4.198429,3.999773
C5IP,0.940531,0.726127,0.497171,0.260098,2.791746,2.977599
C5N,0.939366,0.077784,0.062853,0.166612,0.466860,0.315888
C6NP,0.853630,0.803603,0.533017,0.199284,4.032448,2.100465
C6IP,0.906737,0.842640,0.538920,0.190617,4.420586,2.759222
C6N,0.675390,1.029010,0.683576,0.170446,6.037151,1.806086
C6A,0.765565,0.462827,0.259683,0.451839,1.024319,0.955890
C7NP,0.749125,0.425616,0.320911,0.078346,5.432499,0.849746
C7IP,0.526207,1.076439,0.738825,0.189356,5.684734,1.563851
C7N,0.852422,2.264365,1.587923,0.145409,15.572338,5.894347


In [9]:
modeling(XGBRegressor(learning_rate=0.1,n_estimators=100,objective='reg:squarederror'),train,test)

,R2,RMSE,mean_absolute_error,norm_RMSE,y_mean,std
C5NP,0.972130,0.667739,0.501192,0.159045,4.198429,3.999773
C5IP,0.922363,0.829661,0.454158,0.297184,2.791746,2.977599
C5N,0.906166,0.096764,0.060549,0.207265,0.466860,0.315888
C6NP,0.875072,0.742413,0.512224,0.184110,4.032448,2.100465
C6IP,0.899156,0.876217,0.568755,0.198213,4.420586,2.759222
C6N,0.838017,0.726899,0.479339,0.120404,6.037151,1.806086
C6A,0.358489,0.765614,0.372174,0.747437,1.024319,0.955890
C7NP,0.669552,0.488473,0.345215,0.089917,5.432499,0.849746
C7IP,0.663602,0.907031,0.600430,0.159555,5.684734,1.563851
C7N,0.926743,1.595362,1.171603,0.102448,15.572338,5.894347


In [10]:
modeling(LGBMRegressor(learning_rate=0.1, n_estimators=100),train,test)

,R2,RMSE,mean_absolute_error,norm_RMSE,y_mean,std
C5NP,0.967777,0.717988,0.535394,0.171013,4.198429,3.999773
C5IP,0.938801,0.736614,0.463928,0.263854,2.791746,2.977599
C5N,0.928163,0.084666,0.060210,0.181351,0.466860,0.315888
C6NP,0.856081,0.796846,0.521010,0.197609,4.032448,2.100465
C6IP,0.910050,0.827537,0.524094,0.187201,4.420586,2.759222
C6N,0.799571,0.808572,0.594842,0.133933,6.037151,1.806086
C6A,0.670825,0.548430,0.348555,0.535410,1.024319,0.955890
C7NP,0.636670,0.512200,0.410771,0.094284,5.432499,0.849746
C7IP,0.655572,0.917793,0.632230,0.161449,5.684734,1.563851
C7N,0.926858,1.594117,1.220289,0.102369,15.572338,5.894347


In [11]:
modeling(CatBoostRegressor(learning_rate = 0.1,n_estimators=100,verbose=0),train,test).style.background_gradient()

,R2,RMSE,mean_absolute_error,norm_RMSE,y_mean,std
C5NP,0.976097,0.618395,0.456326,0.147292,4.19843,3.99977
C5IP,0.961036,0.587759,0.379311,0.210535,2.79175,2.9776
C5N,0.94779,0.0721787,0.0522358,0.154605,0.46686,0.315888
C6NP,0.872958,0.748667,0.514092,0.185661,4.03245,2.10047
C6IP,0.912717,0.815175,0.520323,0.184404,4.42059,2.75922
C6N,0.779846,0.847427,0.574684,0.140369,6.03715,1.80609
C6A,0.720287,0.50555,0.324348,0.493548,1.02432,0.95589
C7NP,0.756507,0.419307,0.297058,0.0771849,5.4325,0.849746
C7IP,0.667264,0.902081,0.576438,0.158685,5.68473,1.56385
C7N,0.934714,1.50607,1.09461,0.0967143,15.5723,5.89435


In [12]:
modeling(BayesianRidge(),train,test)

,R2,RMSE,mean_absolute_error,norm_RMSE,y_mean,std
C5NP,0.925197,1.093945,0.829336,0.260561,4.198429,3.999773
C5IP,0.863243,1.101136,0.828167,0.394426,2.791746,2.977599
C5N,0.921639,0.088427,0.072888,0.189407,0.466860,0.315888
C6NP,0.804780,0.928063,0.765386,0.230149,4.032448,2.100465
C6IP,0.897205,0.884653,0.694131,0.200121,4.420586,2.759222
C6N,0.572743,1.180547,0.957741,0.195547,6.037151,1.806086
C6A,0.503410,0.673607,0.518737,0.657615,1.024319,0.955890
C7NP,0.330632,0.695219,0.525400,0.127974,5.432499,0.849746
C7IP,0.170962,1.423910,1.136636,0.250480,5.684734,1.563851
C7N,0.811362,2.560064,2.165265,0.164398,15.572338,5.894347


In [13]:
modeling(LinearRegression(),train,test)

,R2,RMSE,mean_absolute_error,norm_RMSE,y_mean,std
C5NP,0.925165,1.094179,0.828054,0.260616,4.198429,3.999773
C5IP,0.864325,1.096774,0.821347,0.392863,2.791746,2.977599
C5N,0.921955,0.088248,0.072827,0.189024,0.466860,0.315888
C6NP,0.804467,0.928808,0.766950,0.230334,4.032448,2.100465
C6IP,0.896842,0.886214,0.693449,0.200474,4.420586,2.759222
C6N,0.568919,1.185818,0.962151,0.196420,6.037151,1.806086
C6A,0.519769,0.662419,0.514483,0.646692,1.024319,0.955890
C7NP,0.315862,0.702847,0.527808,0.129378,5.432499,0.849746
C7IP,0.168490,1.426032,1.141380,0.250853,5.684734,1.563851
C7N,0.811707,2.557721,2.170301,0.164248,15.572338,5.894347


In [14]:
modeling(Lasso(),train,test)

,R2,RMSE,mean_absolute_error,norm_RMSE,y_mean,std
C5NP,0.858472,1.504723,1.167030,0.358401,4.198429,3.999773
C5IP,0.735981,1.529973,1.290850,0.548034,2.791746,2.977599
C5N,-0.002389,0.316265,0.286369,0.677429,0.466860,0.315888
C6NP,0.590220,1.344593,1.022944,0.333443,4.032448,2.100465
C6IP,0.728366,1.438065,1.147560,0.325311,4.420586,2.759222
C6N,0.337766,1.469753,1.037280,0.243451,6.037151,1.806086
C6A,-0.028161,0.969256,0.762665,0.946244,1.024319,0.955890
C7NP,-0.000208,0.849834,0.696870,0.156435,5.432499,0.849746
C7IP,-0.001557,1.565068,1.251772,0.275311,5.684734,1.563851
C7N,0.714297,3.150596,2.308433,0.202320,15.572338,5.894347


In [15]:
modeling(Ridge(),train,test)

,R2,RMSE,mean_absolute_error,norm_RMSE,y_mean,std
C5NP,0.925203,1.093898,0.829789,0.260549,4.198429,3.999773
C5IP,0.863237,1.101159,0.828202,0.394434,2.791746,2.977599
C5N,0.921448,0.088534,0.072911,0.189638,0.466860,0.315888
C6NP,0.804641,0.928394,0.766071,0.230231,4.032448,2.100465
C6IP,0.897323,0.884145,0.694384,0.200006,4.420586,2.759222
C6N,0.570702,1.183363,0.960173,0.196013,6.037151,1.806086
C6A,0.515417,0.665414,0.515900,0.649616,1.024319,0.955890
C7NP,0.340405,0.690125,0.524095,0.127036,5.432499,0.849746
C7IP,0.171127,1.423769,1.136057,0.250455,5.684734,1.563851
C7N,0.811104,2.561808,2.162745,0.164510,15.572338,5.894347


In [16]:
modeling(MLPRegressor(),train,test)

,R2,RMSE,mean_absolute_error,norm_RMSE,y_mean,std
C5NP,0.932686,1.037735,0.824531,0.247172,4.198429,3.999773
C5IP,0.881264,1.026026,0.748567,0.367521,2.791746,2.977599
C5N,0.939160,0.077916,0.057244,0.166894,0.466860,0.315888
C6NP,0.694558,1.160860,0.915613,0.287880,4.032448,2.100465
C6IP,0.809830,1.203254,0.935333,0.272193,4.420586,2.759222
C6N,0.078472,1.733775,1.348423,0.287184,6.037151,1.806086
C6A,0.661369,0.556252,0.355173,0.543045,1.024319,0.955890
C7NP,-1.850502,1.434663,1.137028,0.264089,5.432499,0.849746
C7IP,-0.359472,1.823395,1.453110,0.320753,5.684734,1.563851
C7N,-0.735430,7.764960,6.699980,0.498638,15.572338,5.894347
